In [ ]:
import findspark
findspark.init('c:/spark')
from pyspark import SparkConf, SparkContext
import collections
import re
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pandas as pd
import collections
from pyspark.ml.linalg import Vectors
from sklearn.cluster import MiniBatchKMeans, KMeans
import numpy as np

In [ ]:
spark = SparkSession.builder.config("spark.sql.warehouse.dir", "C:/temp").appName("SparkSQL").getOrCreate()

In [ ]:
inputLines = spark.sparkContext.textFile("train_full_parsed_clean5.1.csv")

In [ ]:
header=inputLines.filter(lambda l: "_id" in l)

In [ ]:
header.collect()

In [ ]:
linesnoheader = inputLines.subtract(header)

In [7]:
linesnoheader.take(2)

['6.3157195997906035,6.285998094508865,-73.98565673828125,40.7317008972168,-73.99915313720702,40.7443962097168,1813.181999421452,2314.5,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,"[[-73.985631, 40.731737], [-73.989868, 40.733528], [-73.9899, 40.734447], [-74.002491, 40.739745], [-73.999144, 40.744392]]"',
 '6.0748861552470546,6.324358962381311,-73.95960235595702,40.777099609375,-73.9454345703125,40.77409744262695,1239.172234033006,1853.9,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,"[[-73.959587, 40.777094], [-73.960619, 40.775675], [-73.948072, 40.770417], [-73.945393, 40.77408]]"']

In [6]:
coord=pd.read_csv('route_coord_count.csv')
kmeans = MiniBatchKMeans(n_clusters=100, batch_size=10000).fit(coord[['longitude','latitude']])
coord.loc[:, 'cluster'] = kmeans.predict(coord[['longitude','latitude']])

In [7]:
cluster=coord['cluster'].tolist()

cluster_name=['C'+str(i) for i in range(100)]


In [8]:
def topcoordinates(line):
    coordinates=re.findall('(\[\-\d+\.\d+\,\s\d+\.\d+\])',line)
    
    line_cluster=[]
    for i in coordinates:
            temp=re.split('\[|, |\]',i)
            long=float(temp[1])
            lat=float(temp[2])
            cluster_group=kmeans.predict([[long,lat]])
            line_cluster+=[cluster_group[0]]

    line_cluster=list(set(line_cluster))
    all_cluster=[ 1 if i in line_cluster else 0 for i in range(100)]
        
    
    
    
    feature=line.split(',')
    feature1=[float(i) for i in feature[:56]]
    feature2=[int(i) for i in feature1[8:56]]
    feature3=feature1[:8]
    return feature3+feature2+all_cluster

    

In [13]:
a='6.3157195997906035,6.285998094508865,-73.98565673828125,40.7317008972168,-73.99915313720702,40.7443962097168,1813.181999421452,2314.5,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,"[[-73.985631, 40.731737], [-73.989868, 40.733528], [-73.9899, 40.734447], [-74.002491, 40.739745], [-73.999144, 40.744392]]"'
coordinates=re.findall('(\[\-\d+\.\d+\,\s\d+\.\d+\])',a)
line_cluster=[]
for i in coordinates:
        temp=re.split('\[|, |\]',i)
        long=float(temp[1])
        lat=float(temp[2])
        cluster_group=kmeans.predict([[long,lat]])
        line_cluster+=[cluster_group[0]]
        
line_cluster=list(set(line_cluster))
all_cluster=[ 1 if i in line_cluster else 0 for i in range(100)]

In [15]:
line_cluster

[3, 97, 83, 45]

In [9]:
coord = linesnoheader.map(topcoordinates)

In [10]:
feature=['log_duration', 'log_trip_duration', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'great_circle_distance','distance',
       'snow', 'holiday', 'vendor_id', 'pickup_hour_0',
       'pickup_hour_1', 'pickup_hour_2', 'pickup_hour_3', 'pickup_hour_4',
       'pickup_hour_5', 'pickup_hour_6', 'pickup_hour_7', 'pickup_hour_8',
       'pickup_hour_9', 'pickup_hour_10', 'pickup_hour_11',
       'pickup_hour_12', 'pickup_hour_13', 'pickup_hour_14',
       'pickup_hour_15', 'pickup_hour_16', 'pickup_hour_17',
       'pickup_hour_18', 'pickup_hour_19', 'pickup_hour_20',
       'pickup_hour_21', 'pickup_hour_22', 'pickup_hour_23',
       'pickup_weekday_0', 'pickup_weekday_1', 'pickup_weekday_2',
       'pickup_weekday_3', 'pickup_weekday_4', 'pickup_weekday_5',
       'pickup_weekday_6', 'pickup_month_1', 'pickup_month_2',
       'pickup_month_3', 'pickup_month_4', 'pickup_month_5',
       'pickup_month_6', 'passenger_count_0', 'passenger_count_1',
       'passenger_count_2', 'passenger_count_3', 'passenger_count_4',
       'passenger_count_5', 'passenger_count_6', 'passenger_count_7']
colNames=feature+cluster_name

In [11]:
colNames

['log_duration',
 'log_trip_duration',
 'pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude',
 'great_circle_distance',
 'distance',
 'snow',
 'holiday',
 'vendor_id',
 'pickup_hour_0',
 'pickup_hour_1',
 'pickup_hour_2',
 'pickup_hour_3',
 'pickup_hour_4',
 'pickup_hour_5',
 'pickup_hour_6',
 'pickup_hour_7',
 'pickup_hour_8',
 'pickup_hour_9',
 'pickup_hour_10',
 'pickup_hour_11',
 'pickup_hour_12',
 'pickup_hour_13',
 'pickup_hour_14',
 'pickup_hour_15',
 'pickup_hour_16',
 'pickup_hour_17',
 'pickup_hour_18',
 'pickup_hour_19',
 'pickup_hour_20',
 'pickup_hour_21',
 'pickup_hour_22',
 'pickup_hour_23',
 'pickup_weekday_0',
 'pickup_weekday_1',
 'pickup_weekday_2',
 'pickup_weekday_3',
 'pickup_weekday_4',
 'pickup_weekday_5',
 'pickup_weekday_6',
 'pickup_month_1',
 'pickup_month_2',
 'pickup_month_3',
 'pickup_month_4',
 'pickup_month_5',
 'pickup_month_6',
 'passenger_count_0',
 'passenger_count_1',
 'passenger_count_2',
 'passenger_count_3',
 'passen

In [12]:
train = coord.toDF(colNames)

In [ ]:
train.write.csv("train_full_parsed_clean5.csv",header = 'true')

In [13]:
train.repartition(1).write.format('com.databricks.spark.csv').save('csv_cluster.csv',header = 'true')

In [22]:
train.repartition(40).write.parquet('parquet_cluster_40_files')

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import OneHotEncoder

In [ ]:
feature_col=colNames[0:1]+colNames[2:]
vecAssembler = VectorAssembler(inputCols=feature_col, outputCol="features")


In [ ]:
(trainingData, testData) = train.randomSplit([0.8, 0.2],seed=42)

In [ ]:

gbt = GBTRegressor(featuresCol="features",labelCol="log_trip_duration",maxIter=10,maxDepth=10, seed=42)

# Chain indexer and GBT in a Pipeline
pipeline = Pipeline(stages=[vecAssembler,gbt])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

In [ ]:
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.util import MLUtils

model = GradientBoostedTrees.trainRegressor(trainingData,
                                             categoricalFeaturesInfo={}, numIterations=3)